# Train, Validate --> Train, Test

## Introduction
When constructing a model, data availability may become an issue. 
In order to avoid overfitting, it is necessary to withhold some portion of the data as a test set. 
However, overfitting *on the test set* may also occur without a secondary validation step. 
As such, `scikit` contains a number of methods for cross-validation of data.

## References
1. [Scikit documentation - GaussianNB](http://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html)

## Setting up the model

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_iris
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.naive_bayes import GaussianNB
from collections import OrderedDict

# load dataset (we're working with Iris again)
raw = load_iris()
X = raw.data[:, :2] # slice off only the first feature (.data is multi-dimensional)
y = raw.target # the target data is a single label, so it can all be kept

# validation split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)

# we'll use the Gaussian Naive Bayes classifier
classifier = GaussianNB()

## Cross-validation
Though a manual CV workflow was described in [the cross-validation lab](./CrossValidation.ipynb), the automated `cross_val_score()` will work well enough for this example.

In [ ]:
# automated CV step
scores = cross_val_score(classifier, X_train, y_train, cv=5)
print(scores) # TODO: visualization of CV process

These cross-validation values represent how well (with 1 being a perfect score) the model performed against a small, as-yet-untrained portion of the data for the classification task.

## Training the new model
Since the CV values are relatively high, we can proceed:

In [ ]:
# fit new model
fitted = classifier.fit(X_train, y_train)

# GaussianNB.predict() returns class labels (integers)
labels = classifier.predict(X_test)

# pick your favorite colors!
colors = ["red", "blue", "green"]

# iterate over labels and assign color to each point
for i in range(0,len(X_test)):
    col = colors[labels[i]]
    plt.plot(X_test[:,0][i], X_test[:,1][i], color=col, marker='o', 
             markersize=5, label="Class %i" % labels[i])

# cut out duplicate labels
handles, labels = plt.gca().get_legend_handles_labels()
by_label = OrderedDict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys(), loc='upper right')
plt.show()

The plot above - as in the [clustering examples](../resources/clustering.ipynb) seen earlier, 
groups each model-determined class by its index.

## Saving and loading models through pickling / `joblib`

In many cases, the model being trained will be used more than once for the same or different data. 
For large datasets, training can be computationally expensive as well. 
For these and other reasons it is often necessary to save a trained model so it can later be loaded.
This is relatively easy to do from within `scikit` via `joblib`:

In [ ]:
# assuming some model 'clf' is initialized and trained:
from sklearn.externals import joblib
joblib.dump(classifier, 'GaussianIris.pkl')

`joblib` makes loading in pickled models similarly easy:

In [ ]:
# we'll load to a new variable this time, for clarity
loaded_model = joblib.load('GaussianIris.pkl')

# and predict with the 'new' model:
labels = loaded_model.predict(X_test)

# pick your favorite colors!
colors = ["red", "blue", "green"]

# iterate over labels and assign color to each point
for i in range(0,len(X_test)):
    col = colors[labels[i]]
    plt.plot(X_test[:,0][i], X_test[:,1][i], color=col, marker='o', 
             markersize=5, label="Class %i" % labels[i])

# cut out duplicate labels    
handles, labels = plt.gca().get_legend_handles_labels()
by_label = OrderedDict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys(), loc='upper right')
plt.show()

Note that the results are exactly the same.
This is no accident - the model being loaded below is *the same* as the one used above.

Please read a little about Pickling (or Serialization) [here](https://docs.python.org/3/library/pickle.html) and [here](https://en.wikipedia.org/wiki/Serialization).

## Notes on pickling - safety, efficiency

`joblib` (and by extension the default `pickle` module) is by no means your only option for model storage.
`cPickle` is a [faster](https://docs.python.org/2.2/lib/module-cPickle.html) 
C-based implementation of the same pickling algorithm, 
and for more significant models (as we may cover in later modules) it would be worth looking into.

Moreover, consider reading the `scikit` docs on [persistence](http://scikit-learn.org/stable/modules/model_persistence.html)
for considerations on the long-term safety of pickling. 